# Cargill Ocean Transportation Datathon 2026
## Voyage Optimization & Freight Calculator

---

**Team:** [Your Team Name]

**Objective:** Determine optimal vessel-cargo assignments for Cargill's Capesize fleet and committed cargoes to maximize portfolio profit.

---

## 1. Setup & Imports

In [22]:
import os
import sys

# Ensure we're in the correct directory
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Clear any cached imports
for mod in list(sys.modules.keys()):
    if 'portfolio_optimizer' in mod or 'freight_calculator' in mod:
        del sys.modules[mod]

import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend (must be before pyplot import)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Our custom modules
from freight_calculator import (
    FreightCalculator, PortDistanceManager, BunkerPrices,
    Vessel, Cargo, VoyageResult,
    create_cargill_vessels, create_cargill_cargoes,
    create_market_vessels, create_market_cargoes, create_bunker_prices
)
from portfolio_optimizer import (
    PortfolioOptimizer, ScenarioAnalyzer, FullPortfolioOptimizer
)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("Setup complete!")

Setup complete!


## 2. Initialize Calculator & Load Data

In [23]:
# Initialize the freight calculator
distance_mgr = PortDistanceManager('Port_Distances.csv')
bunker_prices = create_bunker_prices()
calculator = FreightCalculator(distance_mgr, bunker_prices)
optimizer = PortfolioOptimizer(calculator)
analyzer = ScenarioAnalyzer(optimizer)
full_optimizer = FullPortfolioOptimizer(calculator)

# Load vessel and cargo data
cargill_vessels = create_cargill_vessels()
cargill_cargoes = create_cargill_cargoes()
market_vessels = create_market_vessels()
market_cargoes = create_market_cargoes()

print(f"Loaded {len(cargill_vessels)} Cargill vessels")
print(f"Loaded {len(cargill_cargoes)} Cargill committed cargoes")
print(f"Loaded {len(market_vessels)} market vessels")
print(f"Loaded {len(market_cargoes)} market cargoes available for bidding")

Loaded 4 Cargill vessels
Loaded 3 Cargill committed cargoes
Loaded 11 market vessels
Loaded 8 market cargoes available for bidding


## 3. Fleet Overview

In [24]:
# Display Cargill vessels
vessel_data = []
for v in cargill_vessels:
    vessel_data.append({
        'Vessel': v.name,
        'DWT (MT)': f"{v.dwt:,}",
        'Hire Rate ($/day)': f"${v.hire_rate:,}",
        'Current Port': v.current_port,
        'ETD': v.etd,
        'VLSFO ROB (MT)': v.bunker_rob_vlsfo,
        'Speed Laden (kn)': v.speed_laden,
        'Speed Ballast (kn)': v.speed_ballast,
    })

print("\n[SHIP] CARGILL VESSELS")
print("=" * 80)
pd.DataFrame(vessel_data)


[SHIP] CARGILL VESSELS


,Vessel,DWT (MT),Hire Rate ($/day),Current Port,ETD,VLSFO ROB (MT),Speed Laden (kn),Speed Ballast (kn)
0,ANN BELL,"180,803","$11,750",QINGDAO,25 Feb 2026,401.3,13.5,14.5
1,OCEAN HORIZON,"181,550","$15,750",MAP TA PHUT,1 Mar 2026,265.8,13.8,14.8
2,PACIFIC GLORY,"182,320","$14,800",GWANGYANG,10 Mar 2026,601.9,13.5,14.2
3,GOLDEN ASCENT,"179,965","$13,950",FANGCHENG,8 Mar 2026,793.3,13.0,14.0


In [25]:
# Display Cargill cargoes
cargo_data = []
for c in cargill_cargoes:
    cargo_data.append({
        'Cargo': c.name[:35],
        'Customer': c.customer,
        'Commodity': c.commodity,
        'Quantity (MT)': f"{c.quantity:,}",
        'Laycan': f"{c.laycan_start} - {c.laycan_end}",
        'Freight ($/MT)': f"${c.freight_rate:.2f}",
        'Load Port': c.load_port[:20],
        'Discharge Port': c.discharge_port,
    })

print("\n[PKG] CARGILL COMMITTED CARGOES")
print("=" * 80)
pd.DataFrame(cargo_data)


[PKG] CARGILL COMMITTED CARGOES


,Cargo,Customer,Commodity,Quantity (MT),Laycan,Freight ($/MT),Load Port,Discharge Port
0,EGA Bauxite (Guinea-China),EGA,Bauxite,"180,000",2 Apr 2026 - 10 Apr 2026,$23.00,KAMSAR ANCHORAGE,QINGDAO
1,BHP Iron Ore (Australia-China),BHP,Iron Ore,"160,000",7 Mar 2026 - 11 Mar 2026,$9.00,PORT HEDLAND,LIANYUNGANG
2,CSN Iron Ore (Brazil-China),CSN,Iron Ore,"180,000",1 Apr 2026 - 8 Apr 2026,$22.30,ITAGUAI,QINGDAO


## 4. Voyage Analysis - All Combinations

In [26]:
# Calculate all voyage combinations
all_voyages = optimizer.calculate_all_voyages(cargill_vessels, cargill_cargoes)

# Create summary view
summary = all_voyages[['vessel', 'cargo', 'can_make_laycan', 'total_days', 
                       'net_freight', 'total_bunker_cost', 'net_profit', 'tce']].copy()
summary['cargo'] = summary['cargo'].str[:30]
summary['net_freight'] = summary['net_freight'].apply(lambda x: f"${x:,.0f}" if pd.notna(x) else 'N/A')
summary['total_bunker_cost'] = summary['total_bunker_cost'].apply(lambda x: f"${x:,.0f}" if pd.notna(x) else 'N/A')
summary['net_profit'] = summary['net_profit'].apply(lambda x: f"${x:,.0f}" if pd.notna(x) else 'N/A')
summary['tce'] = summary['tce'].apply(lambda x: f"${x:,.0f}" if pd.notna(x) else 'N/A')

print("\n[CHART] ALL VOYAGE COMBINATIONS")
print("=" * 100)
summary


[CHART] ALL VOYAGE COMBINATIONS


,vessel,cargo,can_make_laycan,total_days,net_freight,total_bunker_cost,net_profit,tce
0,ANN BELL,EGA Bauxite (Guinea-China),True,89.71,"$4,026,994","$1,550,597","$1,407,338","$27,438"
1,ANN BELL,BHP Iron Ore (Australia-China),True,33.52,"$1,524,600","$514,548","$221,198","$18,349"
2,ANN BELL,CSN Iron Ore (Brazil-China),True,87.50,"$3,805,587","$1,578,944","$1,018,508","$23,390"
3,OCEAN HORIZON,EGA Bauxite (Guinea-China),True,83.74,"$4,043,961","$1,445,564","$1,264,492","$30,850"
4,OCEAN HORIZON,BHP Iron Ore (Australia-China),True,30.69,"$1,524,600","$468,125","$178,078","$21,552"
5,OCEAN HORIZON,CSN Iron Ore (Brazil-China),False,89.36,"$3,821,621","$1,650,341","$583,826","$22,283"
6,PACIFIC GLORY,EGA Bauxite (Guinea-China),False,89.83,"$4,061,449","$1,616,938","$1,099,954","$27,044"
7,PACIFIC GLORY,BHP Iron Ore (Australia-China),False,33.07,"$1,524,600","$526,736","$113,408","$18,229"
8,PACIFIC GLORY,CSN Iron Ore (Brazil-China),False,87.74,"$3,838,148","$1,649,658","$709,936","$22,891"
9,GOLDEN ASCENT,EGA Bauxite (Guinea-China),False,87.53,"$4,007,961","$1,476,742","$1,295,137","$28,746"


In [27]:
# TCE Heatmap
pivot_tce = all_voyages.pivot_table(
    index='vessel',
    columns='cargo',
    values='tce',
    aggfunc='first'
)
pivot_tce.columns = [c[:20] for c in pivot_tce.columns]

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_tce, annot=True, fmt=',.0f', cmap='RdYlGn', 
            cbar_kws={'label': 'TCE ($/day)'})
plt.title('Time Charter Equivalent (TCE) by Vessel-Cargo Combination', fontsize=14, fontweight='bold')
plt.xlabel('Cargo')
plt.ylabel('Vessel')
plt.tight_layout()
plt.savefig('tce_heatmap.png', dpi=150)
plt.show()

In [28]:
# Laycan Feasibility Matrix
pivot_laycan = all_voyages.pivot_table(
    index='vessel',
    columns='cargo',
    values='can_make_laycan',
    aggfunc='first'
)
pivot_laycan.columns = [c[:20] for c in pivot_laycan.columns]

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_laycan.astype(int), annot=pivot_laycan.replace({True: '[OK]', False: '[X]'}), 
            fmt='', cmap=['#ffcccc', '#ccffcc'], cbar=False)
plt.title('Laycan Feasibility Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Cargo')
plt.ylabel('Vessel')
plt.tight_layout()
plt.savefig('laycan_matrix.png', dpi=150)
plt.show()

## 5. Portfolio Optimization

In [29]:
# Optimize vessel-cargo assignments
portfolio = optimizer.optimize_assignments(
    cargill_vessels, 
    cargill_cargoes,
    maximize='profit'
)

print("\n" + "=" * 80)
print("[TROPHY] OPTIMAL PORTFOLIO ASSIGNMENTS")
print("=" * 80)

for vessel, cargo, result in portfolio.assignments:
    print(f"\n[OK] {vessel} -> {cargo}")
    if result:
        print(f"   [DATE] Arrival: {result.arrival_date.strftime('%d %b %Y')} | Laycan ends: {result.laycan_end.strftime('%d %b %Y')}")
        print(f"   [TIME]  Duration: {result.total_days:.1f} days")
        print(f"   [PKG] Cargo: {result.cargo_quantity:,} MT")
        print(f"   [MONEY] TCE: ${result.tce:,.0f}/day")
        print(f"   [UP] Net Profit: ${result.net_profit:,.0f}")

print(f"\n" + "-" * 50)
print(f"[MONEY] TOTAL PORTFOLIO PROFIT: ${portfolio.total_profit:,.0f}")
print(f"[CHART] AVERAGE TCE: ${portfolio.avg_tce:,.0f}/day")


[TROPHY] OPTIMAL PORTFOLIO ASSIGNMENTS

[OK] ANN BELL -> CSN Iron Ore (Brazil-China)
   [DATE] Arrival: 03 Apr 2026 | Laycan ends: 08 Apr 2026
   [TIME]  Duration: 87.5 days
   [PKG] Cargo: 177,303 MT
   [MONEY] TCE: $23,390/day
   [UP] Net Profit: $1,018,508

[OK] OCEAN HORIZON -> EGA Bauxite (Guinea-China)
   [DATE] Arrival: 31 Mar 2026 | Laycan ends: 10 Apr 2026
   [TIME]  Duration: 83.7 days
   [PKG] Cargo: 178,050 MT
   [MONEY] TCE: $30,850/day
   [UP] Net Profit: $1,264,492

--------------------------------------------------
[MONEY] TOTAL PORTFOLIO PROFIT: $2,283,001
[CHART] AVERAGE TCE: $27,120/day


In [30]:
# Show unassigned items
if portfolio.unassigned_cargoes:
    print("\n[!]  UNASSIGNED CARGOES (Need market vessels):")
    for c in portfolio.unassigned_cargoes:
        print(f"   * {c}")

if portfolio.unassigned_vessels:
    print("\n[LIST] AVAILABLE VESSELS (For market cargoes):")
    for v in portfolio.unassigned_vessels:
        print(f"   * {v}")


[!]  UNASSIGNED CARGOES (Need market vessels):
   * BHP Iron Ore (Australia-China)

[LIST] AVAILABLE VESSELS (For market cargoes):
   * PACIFIC GLORY
   * GOLDEN ASCENT


## 6. Scenario Analysis

In [31]:
# Find tipping points
tipping_points = analyzer.find_tipping_points(cargill_vessels, cargill_cargoes)

# Get search limits from the result
max_bunker = tipping_points.get('max_bunker_searched_pct', 100)
max_delay = tipping_points.get('max_delay_searched_days', 20)

print("\n" + "=" * 80)
print("TIPPING POINT ANALYSIS")
print("=" * 80)

if tipping_points['bunker']:
    bp = tipping_points['bunker']
    print(f"\nBUNKER PRICE TIPPING POINT:")
    print(f"   If bunker prices increase by more than {bp['change_pct']:.0f}%,")
    print(f"   the optimal assignment strategy changes.")
else:
    print(f"\nBUNKER PRICE: No tipping point found up to +{max_bunker:.0f}% increase")
    print(f"   Current recommendation is robust to bunker price volatility.")

if tipping_points['port_delay']:
    pd_tp = tipping_points['port_delay']
    print(f"\nPORT DELAY TIPPING POINT:")
    print(f"   If port delays exceed {pd_tp['days']} days,")
    print(f"   the optimal assignment strategy changes.")
else:
    print(f"\nPORT DELAY: No tipping point found up to {max_delay} days")
    print(f"   Current recommendation is robust to port congestion.")


TIPPING POINT ANALYSIS

BUNKER PRICE TIPPING POINT:
   If bunker prices increase by more than 60%,
   the optimal assignment strategy changes.

PORT DELAY: No tipping point found up to 20 days
   Current recommendation is robust to port congestion.


In [32]:
# Bunker price sensitivity chart
bunker_analysis = analyzer.analyze_bunker_sensitivity(
    cargill_vessels, cargill_cargoes,
    price_range=(0.8, 1.4),
    steps=13
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Profit chart
ax1.plot(bunker_analysis['bunker_change_pct'], bunker_analysis['total_profit'] / 1e6, 
         marker='o', linewidth=2, color='#2ecc71')
ax1.axvline(x=0, color='red', linestyle='--', alpha=0.5, label='Baseline')
ax1.set_xlabel('Bunker Price Change (%)', fontsize=12)
ax1.set_ylabel('Total Profit ($ Million)', fontsize=12)
ax1.set_title('Portfolio Profit vs Bunker Price', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# TCE chart
ax2.plot(bunker_analysis['bunker_change_pct'], bunker_analysis['avg_tce'], 
         marker='s', linewidth=2, color='#3498db')
ax2.axvline(x=0, color='red', linestyle='--', alpha=0.5, label='Baseline')
ax2.set_xlabel('Bunker Price Change (%)', fontsize=12)
ax2.set_ylabel('Average TCE ($/day)', fontsize=12)
ax2.set_title('Average TCE vs Bunker Price', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('bunker_sensitivity.png', dpi=150)
plt.show()

In [33]:
# Port delay sensitivity chart
delay_analysis = analyzer.analyze_port_delay_sensitivity(
    cargill_vessels, cargill_cargoes,
    max_delay_days=14
)

fig, ax = plt.subplots(figsize=(10, 5))

ax.fill_between(delay_analysis['port_delay_days'], 
                delay_analysis['total_profit'] / 1e6,
                alpha=0.3, color='#e74c3c')
ax.plot(delay_analysis['port_delay_days'], delay_analysis['total_profit'] / 1e6, 
        marker='o', linewidth=2, color='#e74c3c')
ax.set_xlabel('Additional Port Delay (Days)', fontsize=12)
ax.set_ylabel('Total Profit ($ Million)', fontsize=12)
ax.set_title('Portfolio Profit vs Port Delay (China Ports)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Add cost per day annotation
profit_loss_per_day = (delay_analysis['total_profit'].iloc[0] - delay_analysis['total_profit'].iloc[-1]) / 14
ax.annotate(f'Cost: ~${profit_loss_per_day/1000:,.0f}K per delay day', 
            xy=(7, delay_analysis['total_profit'].mean() / 1e6),
            fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('delay_sensitivity.png', dpi=150)
plt.show()

## 7. Market Vessel Analysis (For Unassigned BHP Cargo)

In [34]:
# Since BHP cargo is unassigned, analyze market vessels that could carry it
bhp_cargo = [c for c in cargill_cargoes if 'BHP' in c.name][0]

print("\n" + "=" * 80)
print("[SHIP] MARKET VESSEL ANALYSIS FOR BHP IRON ORE CARGO")
print("=" * 80)
print(f"\nCargo: {bhp_cargo.name}")
print(f"Laycan: {bhp_cargo.laycan_start} - {bhp_cargo.laycan_end}")
print(f"Load Port: {bhp_cargo.load_port}")

# Calculate voyage for market vessels
market_results = []
for vessel in market_vessels:
    try:
        result = calculator.calculate_voyage(vessel, bhp_cargo, use_eco_speed=True)
        market_results.append({
            'Vessel': vessel.name,
            'Current Port': vessel.current_port,
            'ETD': vessel.etd,
            'Can Make Laycan': '[OK]' if result.can_make_laycan else '[X]',
            'Arrival': result.arrival_date.strftime('%d %b'),
            'Voyage Days': f"{result.total_days:.1f}",
            'TCE ($/day)': f"${result.tce:,.0f}",
            'result': result
        })
    except Exception as e:
        market_results.append({
            'Vessel': vessel.name,
            'Current Port': vessel.current_port,
            'ETD': vessel.etd,
            'Can Make Laycan': '[X]',
            'Error': str(e)[:30]
        })

market_df = pd.DataFrame(market_results)
display_cols = [c for c in market_df.columns if c != 'result']
market_df[display_cols]


[SHIP] MARKET VESSEL ANALYSIS FOR BHP IRON ORE CARGO

Cargo: BHP Iron Ore (Australia-China)
Laycan: 7 Mar 2026 - 11 Mar 2026
Load Port: PORT HEDLAND


,Vessel,Current Port,ETD,Can Make Laycan,Arrival,Voyage Days,TCE ($/day)
0,ATLANTIC FORTUNE,PARADIP,2 Mar 2026,[X],11 Mar,31.2,"$20,791"
1,PACIFIC VANGUARD,CAOFEIDIAN,26 Feb 2026,[OK],10 Mar,34.5,"$17,304"
2,CORAL EMPEROR,ROTTERDAM,5 Mar 2026,[X],12 Apr,60.9,"$1,791"
3,EVEREST OCEAN,XIAMEN,3 Mar 2026,[X],12 Mar,30.6,"$21,664"
4,POLARIS SPIRIT,KANDLA,28 Feb 2026,[X],15 Mar,37.1,"$14,001"
5,IRON CENTURY,PORT TALBOT,9 Mar 2026,[X],16 Apr,60.2,"$1,573"
6,MOUNTAIN TRADER,GWANGYANG,6 Mar 2026,[X],17 Mar,33.3,"$18,640"
7,NAVIS PRIDE,MUNDRA,27 Feb 2026,[X],14 Mar,36.7,"$14,668"
8,AURORA SKY,JINGTANG,4 Mar 2026,[X],15 Mar,33.8,"$18,255"
9,ZENITH GLORY,VIZAG,7 Mar 2026,[X],20 Mar,35.0,"$16,303"


In [35]:
# Find best market vessel for BHP cargo
valid_market = [r for r in market_results if r.get('Can Make Laycan') == '[OK]' and 'result' in r]

if valid_market:
    best = max(valid_market, key=lambda x: x['result'].tce)
    print(f"\n[TROPHY] RECOMMENDED MARKET VESSEL FOR BHP CARGO:")
    print(f"   Vessel: {best['Vessel']}")
    print(f"   TCE: {best['TCE ($/day)']}")
    print(f"   Arrival: {best['Arrival']} (Laycan ends: {bhp_cargo.laycan_end})")
else:
    print("\n[!] No market vessels can make the BHP cargo laycan!")


[TROPHY] RECOMMENDED MARKET VESSEL FOR BHP CARGO:
   Vessel: PACIFIC VANGUARD
   TCE: $17,304
   Arrival: 10 Mar (Laycan ends: 11 Mar 2026)


## 8. Full Portfolio Optimization

Now we optimize across ALL valid combinations:
1. **Cargill vessels -> Cargill cargoes** (committed obligations)
2. **Cargill vessels -> Market cargoes** (bidding opportunities)
3. **Market vessels -> Cargill cargoes** (hire to cover commitments)

Note: Market vessels on market cargoes is NOT a valid combination for Cargill.

In [36]:
# Run full portfolio optimization
full_result = full_optimizer.optimize_full_portfolio(
    cargill_vessels=cargill_vessels,
    market_vessels=market_vessels,
    cargill_cargoes=cargill_cargoes,
    market_cargoes=market_cargoes,
    target_tce=18000,  # Target TCE for market cargo bids
)

print("=" * 80)
print("FULL PORTFOLIO OPTIMIZATION RESULTS")
print("=" * 80)

print("\n--- CARGILL VESSEL ASSIGNMENTS ---")
for vessel_name, cargo_name, option in full_result.cargill_vessel_assignments:
    cargo_type = "Cargill" if any(cargo_name == c.name for c in cargill_cargoes) else "Market"
    print(f"\n  {vessel_name} -> {cargo_name[:40]}")
    print(f"    Type: {cargo_type} cargo")
    print(f"    Profit: ${option.net_profit:,.0f}")
    print(f"    TCE: ${option.tce:,.0f}/day")
    if cargo_type == "Market":
        print(f"    Min Freight Bid: ${option.min_freight_bid:.2f}/MT")

print("\n--- MARKET VESSEL ASSIGNMENTS (for Cargill cargoes) ---")
if full_result.market_vessel_assignments:
    for vessel_name, cargo_name, option in full_result.market_vessel_assignments:
        print(f"\n  {vessel_name} (Market) -> {cargo_name[:40]}")
        print(f"    Recommended Max Hire: ${option.recommended_hire_rate:,.0f}/day")
        print(f"    Voyage Duration: {option.voyage_days:.1f} days")
else:
    print("  None required")

print("\n--- UNASSIGNED ---")
print(f"  Cargill cargoes without coverage: {full_result.unassigned_cargill_cargoes or 'None'}")
print(f"  Cargill vessels without cargo: {full_result.unassigned_cargill_vessels or 'None'}")

print("\n" + "=" * 80)
print(f"TOTAL PORTFOLIO PROFIT: ${full_result.total_profit:,.0f}")
print("=" * 80)

FULL PORTFOLIO OPTIMIZATION RESULTS

--- CARGILL VESSEL ASSIGNMENTS ---

  ANN BELL -> CSN Iron Ore (Brazil-China)
    Type: Cargill cargo
    Profit: $1,018,508
    TCE: $23,390/day

  OCEAN HORIZON -> EGA Bauxite (Guinea-China)
    Type: Cargill cargo
    Profit: $1,264,492
    TCE: $30,850/day

  GOLDEN ASCENT -> Adaro Coal (Indonesia-India)
    Type: Market cargo
    Profit: $1,204,055
    TCE: $36,112/day
    Min Freight Bid: $13.59/MT

--- MARKET VESSEL ASSIGNMENTS (for Cargill cargoes) ---

  PACIFIC VANGUARD (Market) -> BHP Iron Ore (Australia-China)
    Recommended Max Hire: $0/day
    Voyage Duration: 34.5 days

--- UNASSIGNED ---
  Cargill cargoes without coverage: None
  Cargill vessels without cargo: ['PACIFIC GLORY']

TOTAL PORTFOLIO PROFIT: $4,084,151


In [37]:
# Show optimization decision analysis
print("=" * 80)
print("OPTIMIZATION DECISION BREAKDOWN")
print("=" * 80)

# Get specific vessels and cargoes for comparison
ann = [v for v in cargill_vessels if v.name == 'ANN BELL'][0]
ocean = [v for v in cargill_vessels if v.name == 'OCEAN HORIZON'][0]
ega = [c for c in cargill_cargoes if 'EGA' in c.name][0]
bhp = [c for c in cargill_cargoes if 'BHP' in c.name][0]
csn = [c for c in cargill_cargoes if 'CSN' in c.name][0]

# Calculate all possible assignments
r_ann_ega = calculator.calculate_voyage(ann, ega, use_eco_speed=True)
r_ann_csn = calculator.calculate_voyage(ann, csn, use_eco_speed=True)
r_ocean_ega = calculator.calculate_voyage(ocean, ega, use_eco_speed=True)
r_ocean_bhp = calculator.calculate_voyage(ocean, bhp, use_eco_speed=True)

print("\nCargill Vessel Options (only 2 vessels can make laycans):")
print(f"  ANN BELL -> EGA:      ${r_ann_ega.net_profit:>10,.0f}")
print(f"  ANN BELL -> CSN:      ${r_ann_csn.net_profit:>10,.0f}")
print(f"  OCEAN HORIZON -> EGA: ${r_ocean_ega.net_profit:>10,.0f}")
print(f"  OCEAN HORIZON -> BHP: ${r_ocean_bhp.net_profit:>10,.0f}")

opt_a = r_ann_ega.net_profit + r_ocean_bhp.net_profit
opt_b = r_ann_csn.net_profit + r_ocean_ega.net_profit

print("\nCoverage Options (all 3 Cargill cargoes must be served):")
print(f"  Option A: ANN->EGA + OCEAN->BHP = ${opt_a:>12,.0f} (CSN needs market vessel)")
print(f"  Option B: ANN->CSN + OCEAN->EGA = ${opt_b:>12,.0f} (BHP needs market vessel)")

winner = "B" if opt_b > opt_a else "A"
diff = abs(opt_b - opt_a)
print(f"\n  --> OPTION {winner} is better by ${diff:,.0f}")

print("\nWhy this matters:")
print("  - The greedy approach would pick ANN->EGA (highest single profit)")
print("  - But total portfolio profit is maximized by Option B")
print("  - This demonstrates the value of holistic optimization")

OPTIMIZATION DECISION BREAKDOWN

Cargill Vessel Options (only 2 vessels can make laycans):
  ANN BELL -> EGA:      $ 1,407,338
  ANN BELL -> CSN:      $ 1,018,508
  OCEAN HORIZON -> EGA: $ 1,264,492
  OCEAN HORIZON -> BHP: $   178,078

Coverage Options (all 3 Cargill cargoes must be served):
  Option A: ANN->EGA + OCEAN->BHP = $   1,585,416 (CSN needs market vessel)
  Option B: ANN->CSN + OCEAN->EGA = $   2,283,001 (BHP needs market vessel)

  --> OPTION B is better by $697,585

Why this matters:
  - The greedy approach would pick ANN->EGA (highest single profit)
  - But total portfolio profit is maximized by Option B
  - This demonstrates the value of holistic optimization


## 9. Final Recommendation Summary

In [38]:
print("\n" + "=" * 80)
print("EXECUTIVE SUMMARY - FULL PORTFOLIO RECOMMENDATIONS")
print("=" * 80)

print("\n" + "-" * 60)
print("CARGILL VESSEL ASSIGNMENTS")
print("-" * 60)

cargill_profit = 0
for vessel_name, cargo_name, option in full_result.cargill_vessel_assignments:
    cargo_type = "Cargill" if any(cargo_name == c.name for c in cargill_cargoes) else "Market"
    print(f"\n{vessel_name} -> {cargo_name[:45]}")
    print(f"  Type: {cargo_type} cargo")
    print(f"  TCE: ${option.tce:,.0f}/day")
    print(f"  Net Profit: ${option.net_profit:,.0f}")
    print(f"  Duration: {option.voyage_days:.0f} days")
    if cargo_type == "Market":
        print(f"  Min Freight Bid: ${option.min_freight_bid:.2f}/MT")
    cargill_profit += option.net_profit

print("\n" + "-" * 60)
print("MARKET VESSEL HIRE (for Cargill cargoes)")
print("-" * 60)

if full_result.market_vessel_assignments:
    for vessel_name, cargo_name, option in full_result.market_vessel_assignments:
        print(f"\n{vessel_name} (Market) -> {cargo_name[:35]}")
        print(f"  Voyage Duration: {option.voyage_days:.1f} days")
        print(f"  Max Hire Rate: ${option.recommended_hire_rate:,.0f}/day")
else:
    print("  None required - all Cargill cargoes covered by own fleet")

print("\n" + "-" * 60)
print("UNASSIGNED ASSETS")
print("-" * 60)
if full_result.unassigned_cargill_vessels:
    print(f"  Vessels: {', '.join(full_result.unassigned_cargill_vessels)}")
else:
    print("  Vessels: All assigned")
if full_result.unassigned_cargill_cargoes:
    print(f"  Cargoes: {', '.join(full_result.unassigned_cargill_cargoes)}")
else:
    print("  Cargoes: All covered")

print("\n" + "=" * 80)
print(f"TOTAL PORTFOLIO PROFIT: ${full_result.total_profit:,.0f}")
print("=" * 80)

print("\n" + "-" * 60)
print("KEY INSIGHTS")
print("-" * 60)
print("""
1. Only 2 of 4 Cargill vessels can meet committed cargo laycans due to positioning
2. Option B (ANN->CSN + OCEAN->EGA) is $697K more profitable than Option A
3. BHP Iron Ore requires hiring market vessel PACIFIC VANGUARD (only vessel that fits)
4. GOLDEN ASCENT generates additional profit by bidding on Adaro Coal market cargo
5. PACIFIC GLORY has no viable cargo assignment due to late ETD (Mar 10)
6. Recommendation is robust to bunker price increases up to +60%
""")

print("-" * 60)
print("RISK FACTORS")
print("-" * 60)
print("""
- Weather: Long-haul routes (Brazil, Guinea) exposed to weather delays
- Congestion: Chinese ports showing increased waiting times
- Bunker: Prices volatile due to geopolitical factors
- BHP Laycan: Very tight window (Mar 7-11), only 1 market vessel qualifies
""")


EXECUTIVE SUMMARY - FULL PORTFOLIO RECOMMENDATIONS

------------------------------------------------------------
CARGILL VESSEL ASSIGNMENTS
------------------------------------------------------------

ANN BELL -> CSN Iron Ore (Brazil-China)
  Type: Cargill cargo
  TCE: $23,390/day
  Net Profit: $1,018,508
  Duration: 88 days

OCEAN HORIZON -> EGA Bauxite (Guinea-China)
  Type: Cargill cargo
  TCE: $30,850/day
  Net Profit: $1,264,492
  Duration: 84 days

GOLDEN ASCENT -> Adaro Coal (Indonesia-India)
  Type: Market cargo
  TCE: $36,112/day
  Net Profit: $1,204,055
  Duration: 54 days
  Min Freight Bid: $13.59/MT

------------------------------------------------------------
MARKET VESSEL HIRE (for Cargill cargoes)
------------------------------------------------------------

PACIFIC VANGUARD (Market) -> BHP Iron Ore (Australia-China)
  Voyage Duration: 34.5 days
  Max Hire Rate: $0/day

------------------------------------------------------------
UNASSIGNED ASSETS
---------------------

---

## Appendix: Key Assumptions

| Parameter | Assumption | Source |
|-----------|------------|--------|
| Speed Mode | Economical (lower fuel consumption) | Industry standard for cost optimization |
| Bunker Prices | March 2026 forward curve | Datathon provided data |
| Port Working Hours | PWWD SHINC basis | Cargo terms |
| Weather Allowance | Not included in base case | Scenario analysis covers delays |
| Misc Costs | $15,000 per voyage | Industry estimate |
| Cargo Tolerance | Maximum within vessel DWT | Owners' option exercised |
| Target TCE (market bids) | $18,000/day | FFA market reference |

## Valid Optimization Combinations

| Vessel Type | Cargo Type | Allowed | Purpose |
|-------------|------------|---------|---------|
| Cargill | Cargill | Yes | Fulfill committed obligations |
| Cargill | Market | Yes | Bid for additional revenue |
| Market | Cargill | Yes | Hire to cover commitments |
| Market | Market | No | Not Cargill's business |

---

*Report generated using Python-based Freight Calculator and Full Portfolio Optimizer*